<a href="https://colab.research.google.com/github/Ephrem-ETH/Data-Mining-Projects/blob/master/DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stop_words
import nltk
nltk.download('stopwords')

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32919 sha256=e35672ac7b26dc2b74450a3ef83fe532ea8d3e0ef83485a50686e7b616696f97
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install apyori
!pip install preprocessor
!pip install tweet_preprocessor

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5977 sha256=7d3055df32ba51cabd00076dab10e967baa5ecde435be20a050af3725ab74598
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori
  Created wheel for preprocessor: filename=preprocessor-1.1.3-cp36-none-any.whl size=4478 sha256=cd0edce676256ea78e48c816093cf11be51ff94c5d4422a4f3672feb48343bd1
  Stored in directory: /root/.cache/pip/wheels/98/c1/a2/21fbcfd80d76576bbf148991a66f00730f541f265c7600000f
Successfully built preprocessor


In [ ]:
# import required libraries
import warnings
#from mlxtend.frequent_patterns import apriori
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn_pandas import DataFrameMapper
from stop_words import get_stop_words
from apyori import apriori
from datetime import datetime
from collections import Counter
# from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
import numpy as np
import preprocessor as p
import time
import re
import string
import nltk
nltk.download('punkt')
nltk.download('wordnet')
warnings.filterwarnings("ignore")

stop_words = list(get_stop_words('en'))  # About 900 stopwords
nltk_words = list(stopwords.words('english'))
stop_words.extend(nltk_words)
# Global Parameter
# stop_words = stopwords.words('english')
# I see that some stop words are not removed and added manually in the stop words list
stop_words.extend(['what', 'that', 'the', 'are', 'I','have','when', 'If', 'how', 'By', 'Ok', 'aint', 'My', 'not', 'via', 'today', 'TN', 'As', 'In', 'Im', 'from',
                   'that', 'this', 'there', 'A', 'it', 'were', 'We', 'our', 'we', 'didnt', 'each', 'us', 'hey', 'one', 'AS', 'you', 'will', 'wouldnt', 'It', 'K', 'Id'])


# Load the data sets
def load_dataset(filename):
    dataset = pd.read_csv(filename)
    return dataset

# remove unwanted columns


def take_wanted_cols(dataset, cols):
    dataset = dataset[cols]
    return dataset

# Preprocess the tweet by preprocessing

def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)

    return text


def preprocess_tweet_text(tweet):
    tweet.lower()
    # remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#', '', tweet)
    tweet = re.sub(r'\b\w{1,3}\b', '', tweet)
    # Remove numbers
    tweet = re.sub(r'\d+', '', tweet)
    # remove whitespace
    tweet = tweet.strip()
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
  
    return " ".join(stemmed_words)



def frequent_itemsets_engine(tweets):

    frequent_itemsets_in_time = apriori(
    tweets, min_support=0.01, min_confidance=0.05, min_lift=3, min_length=2, target="rules")

    return frequent_itemsets_in_time


def frequent_itemsets(data):
    date = data['date'].unique()
    start_date = date[0]
    results = []
    for i in range(3, len(date), 3):
       
        end_date = date[i]
        

        tweet = data.loc[(data['date'] >=
                          start_date) & (data['date'] <= end_date)]['text']
        frequent_terms = frequent_itemsets_engine(tweet.tolist())

        #x = get_feature_vector(data['text'])
        association_rules = list(frequent_terms)
        #print(association_Results)


        results.append([start_date.strftime("%m/%d/%Y"),
                        end_date.strftime("%m/%d/%Y"), association_rules])
        start_date = end_date

    return results

def frequent_topics(data):
  dates = data['date'].unique()
  frequent_itemsets=[]
  frequent_topics_sets = []
  for date in dates:
    tweet = data.loc[(data['date'] == date)] ['text']
    itemsets = frequent_itemsets_engine(tweet.tolist())
    frequent_itemsets.append(list(itemsets))
  for itemsets in frequent_itemsets:
    #rules = itemsets[0]
    for item in itemsets:
      pair= item[0]
      frequent_topics_sets.append(pair)
  counter = Counter(frequent_topics_sets)
  print(counter)






# Load dataset
dataset = load_dataset('/content/drive/MyDrive/dataset/covid19_tweets.csv')
# gain wanted columns from the raw dataset
raw_tweet = take_wanted_cols(dataset, ['text', 'date'])

# Let's look some records
print(raw_tweet.head())
print(raw_tweet.info())

# drop duplicate records
raw_tweet.drop_duplicates(subset='text', inplace=True)
# Convert the date field to date format


# preprocess data
raw_tweet.text = raw_tweet.apply(preprocess_tweet, axis=1)
raw_tweet.text = raw_tweet['text'].apply(preprocess_tweet_text)

# convert date from string  to datetime fromat
raw_tweet['date'] = pd.to_datetime(raw_tweet['date'])


print("here")
# transform tweet text into vector and combine with the date feature
# mapper = DataFrameMapper([
#     ('text', TfidfVectorizer()),
#     ('date', None),
# ])
# vectorize = TfidfVectorizer()
# x = vectorize.fit_transform(raw_tweet)

raw_tweet['date'] = raw_tweet['date'].dt.date
# raw_tweet['day'] = raw_tweet['date'].dt.day
# raw_tweet['month'] = raw_tweet['date'].dt.month

# raw_tweet['twwetsVec'] = list(x.toarray())

# print(raw_tweet.head())
# features = mapper.fit_transform(raw_tweet)

net_tweet = raw_tweet.sort_values('date', ascending=True)

date = pd.to_datetime(net_tweet['date'].unique())
net_tweet.text = net_tweet['text'].apply(word_tokenize)
#dates.sort(key = lambda date: datetime.strptime(date, '%d %b %Y'))
print(date)
net_tweet['text'].replace('',np.nan,inplace=True)
empty = net_tweet['text'].isnull().sum()
print(net_tweet.describe())
# transactionFiles = net_tweet['text'].tolist()
# print(transactionFiles[0:10])
        
# print(empty)

rules = frequent_topics(net_tweet)
output = pd.DataFrame.from_dict(rules, orient='index').reset_index()
print(output.head(5))
print(net_tweet.head(10))
#Print the association rules
# item_list = []
# for term in rules:
#   start_time = term[0]
#   end_time   = term[1]
#   print("From " + start_time + " " + " to " + end_time )
#   print(" ")
  
#   association_rule = term[2]
#   for i  in range(0,len(association_rule)):
#     pair = association_rule[i][0]
#     item_list.append(pair)
#     print(len(item_list))
#     items = [x for x in pair]
#     print("Rule: " + items[0] + " -> " + items[1])
#     print("Support: " + str(association_rule[i][1]))
#     print("Confidence: " + str(association_rule[i][2][0][2]))
#     print("Lift: " + str(association_rule[i][2][0][3]))
#     print("Rules:" + str(association_rule[i][0]))
#   print("=============================================")
#   print(" ")

# c= Counter(item_list)
# print(c)


  
  # terms = [x for x in pair]
  # print("Rule:" + terms[2][0] + "->" + terms[1])
  # print("Support :" + term[3])
  
net_tweet.to_csv('dataset.csv')

# raw_tweet_ByDate = raw_tweet.groupby('date').text.unique()

# print(raw_tweet_ByDate.head(5))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
                                                text                 date
0  If I smelled the scent of hand sanitizers toda...  2020-07-25 12:27:21
1  Hey @Yankees @YankeesPR and @MLB - wouldn't it...  2020-07-25 12:27:17
2  @diane3443 @wdunlap @realDonaldTrump Trump nev...  2020-07-25 12:27:14
3  @brookbanktv The one gift #COVID19 has give me...  2020-07-25 12:27:10
4  25 July : Media Bulletin on Novel #CoronaVirus...  2020-07-25 12:27:08
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179108 entries, 0 to 179107
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    179108 non-null  object
 1   date    179108 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB
None
